In [4]:
!pip install pandas prophet openpyxl matplotlib

import pandas as pd
from prophet import Prophet
import matplotlib.pyplot as plt
from openpyxl import Workbook, load_workbook
from openpyxl.drawing.image import Image as XLImage
from google.colab import files


uploaded = files.upload()

file_name = '/content/forecasted_dry_van_rates (13).xlsx'
df = pd.read_excel(file_name, sheet_name='forecast 2025 values')

df['Date'] = pd.to_datetime(df['Date'])
df = df.rename(columns={'Date': 'ds', 'Actual Dry Van Linehaul': 'y'})
train_data = df.dropna(subset=['y'])

model = Prophet(yearly_seasonality=True)
model.fit(train_data)

# Create future dates
future_dates = pd.date_range(start="2025-06-01", end="2025-12-28", freq="W")
future_df = pd.DataFrame(future_dates, columns=['ds'])

forecast = model.predict(future_df)

# Save forecast plot
plt.figure(figsize=(10, 6))
model.plot(forecast)
plt.title('Dry Van Linehaul Rate Forecast')
plt.xlabel('Date')
plt.ylabel('Dry Van Linehaul Rate')
plt.tight_layout()
forecast_plot_path = '/content/forecast_plot.png'
plt.savefig(forecast_plot_path)
plt.close()

# Save component (seasonality) plot
fig2 = model.plot_components(forecast)
plt.tight_layout()
components_plot_path = '/content/components_plot.png'
fig2.savefig(components_plot_path)
plt.close()

# Combine forecast with original data, including prediction bands
forecasted_values = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].rename(columns={
    'yhat': 'RATE_TOTAL_EST_MEAN',
    'yhat_lower': 'LOWER_BAND',
    'yhat_upper': 'UPPER_BAND'
})

forecasted_values['Dry Van Linehaul Rates'] = forecasted_values['RATE_TOTAL_EST_MEAN'].round(2)
forecasted_values['LOWER_BAND'] = forecasted_values['LOWER_BAND'].round(2)
forecasted_values['UPPER_BAND'] = forecasted_values['UPPER_BAND'].round(2)

# Merge historical and forecasted data
df_with_forecast = pd.concat([df, forecasted_values], ignore_index=True)

# Save data to Excel file
excel_output_path = '/content/forecasted_dry_van_rates_with_bands.xlsx'
df_with_forecast.to_excel(excel_output_path, index=False)

wb = load_workbook(excel_output_path)
ws = wb.create_sheet(title='Forecast Plots')

# forecast plot
img1 = XLImage(forecast_plot_path)
img1.anchor = 'A1'
ws.add_image(img1)

# components plot
img2 = XLImage(components_plot_path)
img2.anchor = 'A30'
ws.add_image(img2)

wb.save(excel_output_path)

files.download(excel_output_path)


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpeoqvy5x6/lbccxcws.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpeoqvy5x6/unkhkjum.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.11/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=54449', 'data', 'file=/tmp/tmpeoqvy5x6/lbccxcws.json', 'init=/tmp/tmpeoqvy5x6/unkhkjum.json', 'output', 'file=/tmp/tmpeoqvy5x6/prophet_modelwzwf0mx7/prophet_model-20250530132844.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:28:44 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


Saving forecasted_dry_van_rates (13).xlsx to forecasted_dry_van_rates (13) (3).xlsx


13:28:44 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<Figure size 1000x600 with 0 Axes>